# Data Exploration of Bike Sharing Dataset

The data imported has gone through some cleaning using a small command line application written in C#. To further determine the cleanliness of the data I want to have a look to see what the longest and shortest trips are and see if there is anything interesting that comes up. I have created two seperate tables to account for the later data with the extra columns.

In [4]:
SELECT MIN(tripduration) 
FROM trips

(1 row affected)

Total execution time: 00:00:00.168

(No column name)
60


In [5]:
SELECT MIN(tripduration) 
FROM newer_trips

(1 row affected)

Total execution time: 00:00:00.620

(No column name)
-233404


We can see that the lowest duration in the trips table is 60 seconds and it looks like we have some troubling data in the newer\_trips table. Let's explore some further by checking how many there are with a duration less than zero seconds.

In [6]:
SELECT COUNT(*) 
FROM newer_trips 
WHERE tripduration < 0

(1 row affected)

Total execution time: 00:00:00.790

(No column name)
10380


That is quite a number, but compare to the total data in the database it's not that much. I will ignore the data that has a duration less than 60 seconds. The metadata from the data, that fed the trips table, has the following to say about trip duration as well: "Trips greater than 24 hours in duration are excluded". We can confirm or deny that with the following query.

In [7]:
SELECT MAX(tripduration), YEAR(starttime) AS Year 
FROM trips 
GROUP BY YEAR(starttime) 
ORDER BY YEAR(starttime)

(7 rows affected)

Total execution time: 00:00:00.619

(No column name),Year
86399,2013
86392,2014
86362,2015
86365,2016
86338,2017
14336400,2018
10628400,2019


There are 86400 seconds in 24 hours. Data from 2018 did not have any README files with metadata. To keep things uniform I will ignore data with a trip duration higher than 86400 seconds as well. 

Let's have a look at the average ride duration, per year, and per user type.

In [8]:
SELECT AVG(cast(tripduration as bigint)), YEAR(starttime) AS Year, usertype 
FROM trips 
WHERE tripduration > 60 AND tripduration < 86401 
GROUP BY YEAR(starttime), usertype 
ORDER BY YEAR(starttime), usertype

(17 rows affected)

Total execution time: 00:00:00.523

(No column name),Year,usertype
1824,2013,Customer
722,2013,Subscriber
1673,2014,Customer
716,2014,Subscriber
1750,2015,Customer
767,2015,Dependent
724,2015,Subscriber
1857,2016,Customer
728,2016,Dependent
722,2016,Subscriber


In [9]:
SELECT AVG(cast(tripduration as bigint)), YEAR(started_at) AS Year, member_casual
FROM newer_trips 
WHERE tripduration > 60 AND tripduration < 86401 
GROUP BY YEAR(started_at), member_casual 
ORDER BY YEAR(started_at), member_casual

(6 rows affected)

Total execution time: 00:00:00.760

(No column name),Year,member_casual
2248,2020,casual
920,2020,member
1634,2021,casual
817,2021,member
1396,2022,casual
699,2022,member


It looks like on an anual basis Customers/Casuals use their bikes longer than annual subscribers, about twice as long. The below query results in a chart where we can see that on an annual basis the use of bikes is very seasonal.

In [10]:
SELECT YEAR(starttime) AS trip_year, MONTH(starttime) AS trip_month, usertype, COUNT(*) AS trip_count
FROM trips 
WHERE tripduration > 60 AND tripduration < 86401 
GROUP BY YEAR(starttime), MONTH(starttime), usertype 
ORDER BY YEAR(starttime), MONTH(starttime), usertype

(180 rows affected)

Total execution time: 00:00:00.806

trip_year,trip_month,usertype,trip_count
2013,6,Customer,3120
2013,6,Subscriber,882
2013,7,Customer,51548
2013,7,Subscriber,23313
2013,8,Customer,110096
2013,8,Subscriber,60397
2013,9,Customer,105325
2013,9,Subscriber,95689
2013,10,Customer,64200
2013,10,Subscriber,110484


In [11]:
SELECT YEAR(started_at) AS trip_year, MONTH(started_at) AS trip_month, member_casual, COUNT(*) AS trip_count
FROM newer_trips 
WHERE tripduration > 60 AND tripduration < 86401 
GROUP BY YEAR(started_at), MONTH(started_at), member_casual 
ORDER BY YEAR(started_at), MONTH(started_at), member_casual

(56 rows affected)

Total execution time: 00:00:00.780

trip_year,trip_month,member_casual,trip_count
2020,1,casual,7678
2020,1,member,134865
2020,2,casual,12206
2020,2,member,125383
2020,3,casual,24418
2020,3,member,114351
2020,4,casual,23361
2020,4,member,60268
2020,5,casual,86031
2020,5,member,111576


To get a better understanding of the seasonal aspect of the data it would be interesting to see the most frequented locations per month. I have created views that look like:

```
SELECT YEAR(starttime) AS trip_year, MONTH(starttime) AS trip_month, COUNT(*) AS trips_made, from_station_name, usertypeFROM trips WHERE tripduration > 60 AND tripduration < 86401 GROUP BY YEAR(starttime), MONTH(starttime), from_station_name, usertype

```
```
SELECT YEAR(started_at) AS trip_year, MONTH(started_at) AS trip_month, COUNT(*) AS trips_made, start_station_name, member_casualFROM newer_trips WHERE tripduration > 60 AND tripduration < 86401 GROUP BY YEAR(started_at), MONTH(started_at), start_station_name, member_casual
```

These views I can then query using the below code as an example to get stats on location usage.

In [12]:
SELECT  *
FROM    
        (
            SELECT trip_year, trip_month, trips_made, from_station_name, usertype,
            DENSE_RANK() OVER(PARTITION BY trip_year, trip_month ORDER BY trips_made DESC) AS rn
            FROM trips_month_location_member
            WHERE usertype = 'Subscriber'
        ) s
WHERE   rn <= 10
ORDER   BY trip_year, trip_month

(801 rows affected)

Total execution time: 00:00:00.669

trip_year,trip_month,trips_made,from_station_name,usertype,rn
2013,6,32,Daley Center Plaza,Subscriber,1
2013,6,30,Clark St & Randolph St,Subscriber,2
2013,6,30,Canal St & Jackson Blvd,Subscriber,2
2013,6,27,Clinton St & Washington Blvd,Subscriber,3
2013,6,26,Franklin St & Jackson Blvd,Subscriber,4
2013,6,25,Wabash Ave & Roosevelt Rd,Subscriber,5
2013,6,24,Michigan Ave & Pearson St,Subscriber,6
2013,6,21,McClurg Ct & Illinois St,Subscriber,7
2013,6,21,Dayton St & North Ave,Subscriber,7
2013,6,20,Franklin St & Chicago Ave,Subscriber,8


In [13]:
SELECT  *
FROM    
        (
            SELECT trip_year, trip_month, trips_made, from_station_name, usertype,
            DENSE_RANK() OVER(PARTITION BY trip_year, trip_month ORDER BY trips_made DESC) AS rn
            FROM trips_month_location_member
            WHERE usertype = 'Customer'
        ) s
WHERE   rn <= 10
ORDER   BY trip_year, trip_month

(821 rows affected)

Total execution time: 00:00:00.369

trip_year,trip_month,trips_made,from_station_name,usertype,rn
2013,6,191,Millennium Park,Customer,1
2013,6,172,Lake Shore Dr & Monroe St,Customer,2
2013,6,170,Michigan Ave & Oak St,Customer,3
2013,6,156,McClurg Ct & Illinois St,Customer,4
2013,6,103,Michigan Ave & Pearson St,Customer,5
2013,6,88,Michigan Ave & Lake St,Customer,6
2013,6,88,Wabash Ave & Roosevelt Rd,Customer,6
2013,6,87,Cannon Dr & Fullerton Ave,Customer,7
2013,6,74,McCormick Place,Customer,8
2013,6,62,Fairbanks Ct & Grand Ave,Customer,9


Looking at a map of Chicago gives a good indication of how bikes are used seasonally by the different customer types. Customers use the bikes for leisure and Subscribers more to get from stations to their destination.

To get more insights I have also created views based on the below code. These will show the usage of bikes drilled down to the days of the week, and I have included the Weeknumber for extra filtering if necessary.

```
SELECT trip_id, starttime, stoptime, bikeid, tripduration, from_station_id, from_station_name, to_station_id, to_station_name, usertype, gender, birthday, (DATEPART(WEEKDAY, starttime) + @@DATEFIRST + 6 - 1) % 7 + 1 AS DayOfWeekStartTimeNumber, DATENAME(WEEKDAY, starttime) AS DayOfWeekStartTimeAt, (DATEPART(WEEKDAY, stoptime) + @@DATEFIRST + 6 - 1) % 7 + 1 AS DayOfWeekStopTimeNumber,                          DATENAME(WEEKDAY, stoptime) AS DayOfWeekStopTime, DATEPART(WEEK, starttime) AS WeekNumber FROM dbo.tripsWHERE tripduration > 60 AND tripduration < 86401 

```
```
SELECT ride_id, started_at, ended_at, rideable_type, tripduration, start_station_id, start_station_name, end_station_id, end_station_name, member_casual, start_lat, start_lng, end_lat, end_lng, (DATEPART(WEEKDAY, started_at) + @@DATEFIRST + 6 - 1) % 7 + 1 AS DayOfWeekStartedAtNumber, DATENAME(WEEKDAY, started_at) AS DayOfWeekStartedAt, (DATEPART(WEEKDAY, ended_at) + @@DATEFIRST + 6 - 1) % 7 + 1 AS DayOfWeekEndedAtNumber, DATENAME(WEEKDAY, ended_at) AS DayOfWeekEndedAt, DATEPART(WEEK, started_at) AS WeekNumber FROM dbo.newer_tripsWHERE tripduration > 60 AND tripduration < 86401 

```